In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk, concatenate_datasets
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments, Trainer
#from transformers.adapters import AdapterTrainer

import pandas as pd
import numpy as np
import evaluate

import random
import math
import time
from tqdm import tqdm
import os
import json                                                     

In [ ]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the Model

In [ ]:
#Load the 0.6 threshold model
experiment = 'en-ha_finetune_base_model-1'
base_model_path = f'./0.7_base_model/{experiment}'
model = M2M100ForConditionalGeneration.from_pretrained(base_model_path).to(device)
tokenizer = M2M100Tokenizer.from_pretrained(base_model_path)



# Prepare Data

In [ ]:
src_lang = 'ha'
tgt_lang = 'en'
tokenizer.src_lang = "ha"
tokenizer.tgt_lang = "en"

In [ ]:
# Load the datasets from Hugging Face Hub
original_train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
original_valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')

new_filtered_train_dataset = original_train_dataset.filter(lambda example: example['score'] > 0.8)
new_filtered_valid_dataset = original_valid_dataset.filter(lambda example: example['score'] > 0.8)
new_filtered_train_dataset = new_filtered_train_dataset.remove_columns(['score', 'ref'])
new_filtered_valid_dataset = new_filtered_valid_dataset.remove_columns(['score', 'ref'])

train_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_train.csv', split='train')
valid_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/cleaned_dev.csv', split='train')

train_dataset = train_dataset.rename_column('ha', 'src')
valid_dataset = valid_dataset.rename_column('ha', 'src')
train_dataset = train_dataset.rename_column('en', 'mt')
valid_dataset = valid_dataset.rename_column('en', 'mt')

new_train_dataset = concatenate_datasets([train_dataset, new_filtered_train_dataset])
new_valid_dataset = concatenate_datasets([valid_dataset, new_filtered_valid_dataset])  
new_train_dataset = new_train_dataset.shuffle(seed=42)
new_valid_dataset = new_valid_dataset.shuffle(seed=42)

In [ ]:
train_dataset

In [ ]:
valid_dataset

In [ ]:
new_train_dataset

In [ ]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
new_tokenized_train_dataset = new_train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)
new_tokenized_valid_dataset = new_valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

# Create the DatasetDict
new_tokenized_dataset = DatasetDict({
    'train': new_tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': new_tokenized_valid_dataset  # Directly assign the processed dataset
})

In [ ]:
new_tokenized_dataset

# Training Setup

In [ ]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics



In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = TrainingArguments(
    f"./lang_adapters/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_tokenized_dataset["train"],
    eval_dataset=new_tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

# Evaluation

In [ ]:
improved_model_path = f'./improved_0.8_base_model/{experiment}'

if not os.path.exists(improved_model_path):
    os.makedirs(improved_model_path)

trainer.save_model(improved_model_path)

In [ ]:
test_dataset = load_dataset('csv', data_files='/home/spandey7/Language-Adapters/Data/en-ha/test.csv', split='train')

#rename 'sentence_eng_Latn' to 'en' and 'sentence_hau_Latn' to 'ha'
test_dataset = test_dataset.rename_column('sentence_eng_Latn','mt')
test_dataset = test_dataset.rename_column('sentence_hau_Latn','src')

#tokenize the dataset
tokenized_test_dataset = test_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src',  'mt']  # Specify the correct columns to remove
)

#convert the dataset into dataset dict
tokenized_test_dataset = DatasetDict({
    'test': tokenized_test_dataset  # Directly assign the processed dataset
})

new_eval_results = trainer.evaluate(tokenized_test_dataset["test"])

In [ ]:
new_eval_results